# ANOVA Testing Evaluation

In [ ]:
from scipy.stats import f_oneway
import pandas as pd

## Ingest Data

In [ ]:
df = pd.read_csv('../prompting_fairness_results.csv')

df['changed_strategy'] = df['changed_strategy'].replace({'none': 'no', 'maybe': 'yes'})
trustee_df = df[df['role'] == 'trustee']
persona_trustee = df[df['persona_used'] == 'yes']

## Cleaning Data for Model Input

In [ ]:
def dummies(dataframe):
    """Get dummies for categorical encoding. 
    
    Returns:
        df: pandas.Dataframe with dummy variables 
        """
    return pd.get_dummies(dataframe)

In [ ]:
regression = df.copy()
target = regression['total_money']
regression = regression.drop(columns= 'total_money')
col_by_dtype = regression.columns.to_series().groupby(regression.dtypes).groups
col_by_dtype = {k.name: v for k, v, in col_by_dtype.items()}
regression[col_by_dtype['object']]
cat_dummies = pd.get_dummies(regression[col_by_dtype['object']])

In [ ]:
persona = persona_trustee.copy()
persona = persona.drop(columns= ['role', 'total_money', 'money_given'])
pers_by_dtype = persona.columns.to_series().groupby(persona.dtypes).groups
pers_by_dtype = {k.name: v for k, v, in pers_by_dtype.items()}
persona_dummies = pd.get_dummies(persona[pers_by_dtype['object']])

In [ ]:
trustee = trustee_df.copy()
target2 = trustee['money_given']
trustee = trustee.drop(columns= ['role', 'total_money', 'money_given'])
trust_by_dtype = trustee.columns.to_series().groupby(trustee.dtypes).groups
trust_by_dtype = {k.name: v for k, v, in trust_by_dtype.items()}
trust_dummies = pd.get_dummies(trustee[trust_by_dtype['object']])

anova_input = pd.concat([target, cat_dummies], axis = 1)
persona_anova = pd.concat([persona_trustee['money_given'], persona_dummies], axis = 1)

In [ ]:
framed_today_oneway = f_oneway(df['total_money'][df['framed'] == 'no'],
         df['total_money'][df['framed'] == 'history'],
         df['total_money'][df['framed'] == 'recipient_need'],
         df['total_money'][df['framed'] == 'recipient_millionaire'])

framed_today_oneway: F_onewayResult(statistic=10.520872299662205, pvalue=4.709405456076698e-06)

In [ ]:
persona_given_oneway = f_oneway(persona_trustee['money_given'][persona_trustee['framed'] == 'recipient_need'],
         persona_trustee['money_given'][persona_trustee['framed'] == 'recipient_millionaire'])
persona_given_oneway

persona_given_oneway: F_onewayResult(statistic=35.32575291948371, pvalue=6.804137621247225e-07)

In [ ]:
trustee_oneway = f_oneway(trustee_df['money_given'][trustee_df['framed'] == 'history'],
         trustee_df['money_given'][trustee_df['framed'] == 'recipient_need'],
         trustee_df['money_given'][trustee_df['framed'] == 'recipient_millionaire'])
trustee_oneway

trustee_oneway: F_onewayResult(statistic=32.3168209845946, pvalue=3.3273829866157587e-10)